# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Neural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-1. rnn_string_sequence
<hr>
``` python 

# Lab 12 RNN
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

X = tf.placeholder(
    tf.float32, [None, sequence_length, hidden_size])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(t_cnt, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

'''
 1th training... 
  loss : 1.6107723712921143, 	 prediction : [[2 3 3 4 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellooo
 2th training... 
  loss : 1.525847315788269, 	 prediction : [[2 3 3 3 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellloo
 3th training... 
  loss : 1.453248381614685, 	 prediction : [[2 3 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elello
 4th training... 
  loss : 1.381018877029419, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 5th training... 
  loss : 1.3161988258361816, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
  
...
...

 1998th training... 
  loss : 0.6351824998855591, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1999th training... 
  loss : 0.6351807117462158, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 2000th training... 
  loss : 0.635178804397583, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
  
'''

```

<hr>

In [3]:
import tensorflow as tf
import numpy as np

C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
word = "여보!안경안보여"
word = list(set(word))
word = sorted(word)
word

['!', '경', '보', '안', '여']

In [5]:
tf.set_random_seed(777)  # reproducibility

idx2char = ['!', '경', '보', '안', '여']
# idx2char = word

# Teach hello: hihell -> ihello
x_data    = [[4, 2, 0, 3, 1, 3, 2]]   # 여보!안경안보

x_one_hot = [[[0, 0, 0, 0, 1],     # 여 4
              [0, 0, 1, 0, 0],     # 보 2
              [1, 0, 0, 0, 0],     # !  0
              [0, 0, 0, 1, 0],     # 안 3
              [0, 1, 0, 0, 0],     # 경 1
              [0, 0, 0, 1, 0],     # 안 3
              [0, 0, 1, 0, 0]]]    # 보 2

y_data = [[2, 0, 3, 1, 3, 2, 4]]   # 보!안경안보여

In [6]:
input_dim       = 5   # one-hot size
hidden_size     = 5   # output from the LSTM. 5 to directly predict one-hot
batch_size      = 1   # one sentence
sequence_length = 7   # len("보!안경안보여") == 7


In [7]:
X = tf.placeholder(
    tf.float32, [None, sequence_length, hidden_size])   # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])   # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [8]:
time1 = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        
        cnt = t_cnt+1
        if cnt<10 or cnt%100==0:
            # print(t_cnt, "loss:", l, "prediction: ", result, "true Y: ", y_data)
            print(" {cnt}th training... \n  Loss : {loss}, Prediction : {pred}, True Y : {y_data} ".format(
                    cnt  = cnt, 
                    loss = l, 
                    pred = result, 
                    y_data = y_data
            ), end="\t→ ")

            # print char using dic
            result_str = [idx2char[c] for c in np.squeeze(result)]
            print("Prediction Str :", ''.join(result_str))
        
time2 = time.time()


 1th training... 
  Loss : 1.607013463973999, Prediction : [[3 4 3 4 3 3 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안여안여안안여
 2th training... 
  Loss : 1.5445698499679565, Prediction : [[3 4 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안여안보안보여
 3th training... 
  Loss : 1.488703966140747, Prediction : [[3 3 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안안안보안보여
 4th training... 
  Loss : 1.4339848756790161, Prediction : [[3 0 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안!안보안보여
 5th training... 
  Loss : 1.3778700828552246, Prediction : [[3 0 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안!안보안보여
 6th training... 
  Loss : 1.3250327110290527, Prediction : [[3 0 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : 안!안보안보여
 7th training... 
  Loss : 1.2718167304992676, Prediction : [[0 0 3 2 3 2 4]], True Y : [[2, 0, 3, 1, 3, 2, 4]] 	→ Prediction Str : !!안보안보여
 8th training... 
  Lo

In [9]:
chk_processting_time(time1, time2)

처리시간 : 0분 4초 경과되었습니다.


4.7917304039001465

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>